In [1]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import pandas as pd
data = pd.read_excel('C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\trying\\DATASET.xlsx')

In [8]:
data.head()
data.tail()

,Thickness(mm),Diameter(mm),Force (N),Max def(mm),Min Stress(Mpa),Max Stress(Mpa),case
46,5,20,96000,0.11008,25.701,627.53,47
47,5,20,97000,0.11122,25.968,634.07,48
48,5,20,98000,0.11237,26.236,640.61,49
49,5,20,99000,0.11352,26.504,647.15,50
50,5,20,100000,0.11466,26.772,653.68,51


In [9]:
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np


image_dir = 'C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\trying\\FORCE'

# lists to store image data and corresponding labels
stress_images = []
deformation_images = []
labels_stress = []
labels_deformation = []

# Iterate dataset and match the images to their force
for idx, row in data.iterrows():
    force = row['Force (N)']
    
    
    stress_image_name = f"S_{int(force)}.jpeg"
    deformation_image_name = f"D_{int(force)}.jpeg"
    
    stress_image_path = os.path.join(image_dir, stress_image_name)
    deformation_image_path = os.path.join(image_dir, deformation_image_name)
    
    # preprocess the stress image
    if os.path.exists(stress_image_path):
        stress_img = load_img(stress_image_path, target_size=(128, 128))  
        stress_img_array = img_to_array(stress_img) / 255.0  
        stress_images.append(stress_img_array)
        labels_stress.append([row['Max Stress(Mpa)'], row['Min Stress(Mpa)']])  
    
    # preprocess the deformation image
    if os.path.exists(deformation_image_path):
        deformation_img = load_img(deformation_image_path, target_size=(128, 128))  
        deformation_img_array = img_to_array(deformation_img) / 255.0  
        deformation_images.append(deformation_img_array)
        labels_deformation.append(row['Max def(mm)'])  


stress_images = np.array(stress_images)
deformation_images = np.array(deformation_images)
labels_stress = np.array(labels_stress)
labels_deformation = np.array(labels_deformation)

# shape
stress_images.shape, deformation_images.shape, labels_stress.shape, labels_deformation.shape


((51, 128, 128, 3), (51, 128, 128, 3), (51, 2), (51,))

In [13]:
data.head()

,Thickness(mm),Diameter(mm),Force (N),Max def(mm),Min Stress(Mpa),Max Stress(Mpa),case
0,5,20,50000,0.057332,13.386,326.84,1
1,5,20,51000,0.058478,13.654,333.38,2
2,5,20,52000,0.059625,13.921,339.91,3
3,5,20,53000,0.060771,14.189,346.45,4
4,5,20,54000,0.061918,14.457,352.99,5


In [10]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


def build_model(output_size):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        Flatten(),
        
        
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        
        
        Dense(output_size, activation='linear')
    ])
    
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])
    return model


stress_model = build_model(output_size=2)


deformation_model = build_model(output_size=1)


stress_model.summary()
deformation_model.summary()


C:\Users\hasan\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,313,026 (12.64 MB)

 Trainable params: 3,313,026 (12.64 MB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,312,961 (12.64 MB)

 Trainable params: 3,312,961 (12.64 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Train stress prediction model
stress_model.fit(stress_images, labels_stress, epochs=10, batch_size=32, validation_split=0.2)

# Train deformation prediction model
deformation_model.fit(deformation_images, labels_deformation, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 542ms/step - loss: 106263.1172 - mae: 236.3577 - val_loss: 185544.7031 - val_mae: 315.4946
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 100374.7734 - mae: 227.6952 - val_loss: 157959.9844 - val_mae: 287.0819
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 79992.4141 - mae: 199.6598 - val_loss: 92444.0078 - val_mae: 219.8716
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - loss: 32846.6875 - mae: 134.4899 - val_loss: 5604.3164 - val_mae: 70.4429
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 14649.9463 - mae: 91.1332 - val_loss: 574.3813 - val_mae: 21.8319
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 37061.8594 - mae: 146.5978 - val_loss: 20525.2559 - val_mae: 109.6719
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 9069.3633 - mae: 75.2734 - val_loss: 53394.3125 - val_mae: 179.2216
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 17322.9141 - mae: 97.6451 - val_loss: 65667.4375 - val_m

In [12]:
# Evaluate stress model
stress_loss, stress_mae = stress_model.evaluate(stress_images, labels_stress)
print(f"Stress Model - Loss: {stress_loss}, MAE: {stress_mae}")

# Evaluate deformation model
deformation_loss, deformation_mae = deformation_model.evaluate(deformation_images, labels_deformation)
print(f"Deformation Model - Loss: {deformation_loss}, MAE: {deformation_mae}")


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - loss: 12451.7822 - mae: 82.1379
Stress Model - Loss: 15751.2255859375, MAE: 92.75872802734375
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - loss: 0.0029 - mae: 0.0512
Deformation Model - Loss: 0.0032889381982386112, MAE: 0.05480185151100159


In [19]:
from PIL import Image
import numpy as np

def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.resize((128, 128))
    image_array = np.array(image)
    image_array = image_array / 255.0  
    image_array = np.expand_dims(image_array, axis=0)
    return image_array



In [23]:
image_path = 'C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\trying\\FORCE\\S_50000.jpeg'  
new_stress_image = preprocess_image(image_path)


predicted_stress = stress_model.predict(new_stress_image)
print(f"Predicted Max/Min Stress: {predicted_stress}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted Max/Min Stress: [[336.092    -19.795008]]


In [26]:
image_path = 'C:\\Users\\hasan\\OneDrive\\Desktop\\jupyter\\trying\\FORCE\\D_50000.jpeg'  
new_deformation_image = preprocess_image(image_path)


predicted_deformation = deformation_model.predict(new_deformation_image)
print(f"Predicted Max Deformation: {predicted_deformation}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Predicted Max Deformation: [[0.02907763]]
